# Import dependencies

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, Dropout, SpatialDropout2D
from keras.optimizers import RMSprop, SGD, Adam
from keras.datasets import mnist
from keras.utils import np_utils
from keras import backend as K

# Load the MNIST Dataset

In [ ]:
img_rows, img_cols = 28, 28         # input image dimensions
nb_classes = 10
number_of_data = 2000

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print('X_train original shape:', X_train.shape)

if K.image_dim_ordering() == 'th':
    # For Theano backend
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    # For TensorFlow backend
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    

# Split train test
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train = X_train[:number_of_data]
Y_train = Y_train[:number_of_data]
X_test = X_test[:number_of_data]
Y_test = Y_test[:number_of_data]

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
    

# Visualize MNIST dataset
import matplotlib.pyplot as plt
%matplotlib inline

n = 10  # how many digits we will display
plt.figure(figsize=(10, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(X_train[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

# Convolutional Neural Network
<img align="left" width="70%" src="images/cnn.png"/>

## Build

In [ ]:
# Settings
pool_size = (2, 2)                  # size of pooling area for max pooling
prob_drop_conv = 0.2                # drop probability for dropout @ conv layer
prob_drop_hidden = 0.5              # drop probability for dropout @ fc layer

# Convolutional model
model = Sequential()

# conv1 layer
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), padding='same'))
model.add(SpatialDropout2D(prob_drop_conv))

# conv2 layer
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), padding='same'))
model.add(Flatten())
model.add(Dropout(prob_drop_conv))

# fc1 layer
model.add(Dense(625, activation='relu'))
model.add(Dropout(prob_drop_hidden))

# fc2 layer
model.add(Dense(10, activation='softmax'))

opt = SGD(lr=0.01)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

## Train

In [ ]:
# Settings
batch_size = 128
epochs = 10

# Train
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, shuffle=True, verbose=1)

## Evaluate

In [ ]:
evaluation = model.evaluate(X_test, Y_test, batch_size=256, verbose=1)
print('Summary: Loss over the test dataset: %.2f, Accuracy: %.2f' % (evaluation[0], evaluation[1]))

# Layers & Hyperparameters

## Preliminaries

In [ ]:
def default_optim():
  return SGD(lr=0.01)

def train_model(model, optim=default_optim(), epochs=10, batch_size=128):
  model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['accuracy'])
  model.summary()
  history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, verbose=1)
  
def eval_model(model):
  evaluation = model.evaluate(X_test, Y_test, verbose=1)
  print('Summary: Loss over the test dataset: %.4f, Accuracy: %.4f' % (evaluation[0], evaluation[1]))
  
def train_and_eval_model(model, optim=default_optim(), epochs=10, batch_size=128):
  train_model(model, optim, epochs, batch_size)
  eval_model(model)

## Convolution Layers

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

train_and_eval_model(model)

## Pooling Layers

### Max Pooling

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), padding='same'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

train_and_eval_model(model)

### Average Pooling

In [ ]:
pool_size = (2,2)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
model.add(AveragePooling2D(pool_size=pool_size, strides=(2,2), padding='same'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

train_and_eval_model(model)

## Dropout Layers

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
model.add(SpatialDropout2D(prob_drop_conv))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

train_and_eval_model(model)